In [ ]:
#
# Prototyping parts of the following:
#
# Drawing Graphs to Convey Proximity: An Incremental Arrangement Method
# J.D. Cohen
# ACM Transactions on Computer-Human Interaction, Vol. 4, No. 3, September 1997, Pages 197–229.
#
import polars as pl
import networkx as nx
import numpy as np

from math import log10, ceil
import random
import time
import rtsvg
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
rt = rtsvg.RACETrack()

from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()

from convey_proximity_layout import ConveyProximityLayout

def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)

#
# Attempting to match Figure 1 from "Drawing Graphs To Convey Proximity"
# - looks like i don't do enough iterations for proportional or semi-proportional
# - absolute stress looks like the papers version
#
_tiles_ = []
g       = _patterns_.createPattern('X')
df      = graphAsDataFrame(g)
for k in range(3):
    _lu_ = {'iteration':[], 'stress':[], 'trial':[], 'time':[]}
    for _trial_ in range(1): # should be 15 to 25
        t0    = time.time()
        pfdl  = rtsvg.PolarsForceDirectedLayout(g, k=k)
        t1    = time.time()
        _vec_ = pfdl.stressVector()
        for i in range(len(_vec_)):
            _lu_['iteration'].append(i), _lu_['stress'].append(log10(_vec_[i])), _lu_['trial'].append(_trial_), _lu_['time'].append(t1-t0)
    df_stress = pl.DataFrame(_lu_)
    _tiles_.append(rt.titleSVG(rt.xy(df_stress, x_field='iteration', y_field='stress', line_groupby_field='trial', dot_size=None), f'k={k}'))
#rt.tile(_tiles_, spacer=10) # uncomment to display

In [ ]:
_tiles_, _cpls_ = [], []
for i in range(len(_patterns_)):
    _pattern_ = _patterns_[i]
    g         = _patterns_.createPattern(_pattern_)
    df        = graphAsDataFrame(g)
    t0 = time.time()
    _link_ = rt.link(df, [('fm','to')], rtsvg.PolarsForceDirectedLayout(g,k=0.0).results())
    t1 = time.time()
    _tiles_.append(rt.titleSVG(_link_, f'PFDL : {t1-t0:0.2f}s'))
    t0 = time.time()
    _link_ = rt.link(df, [('fm','to')], rtsvg.PolarsSpringLayout(g).results())
    t1 = time.time()
    _tiles_.append(rt.titleSVG(_link_, f'PSL : {t1-t0:0.2f}s'))
    for k in range(3):
        t0 = time.time()
        _cpls_.append(ConveyProximityLayout(g, k=k))
        _link_ = rt.link(df, [('fm','to')], _cpls_[-1].results())
        t1 = time.time()
        _tiles_.append(rt.titleSVG(_link_, f'{k=} : {t1-t0:0.2f}s'))
rt.table(_tiles_, per_row=5, spacer=10)

In [ ]:
_tiles_ = []
for i in range(len(_cpls_)):
    df_stress = _cpls_[i].stress_df.with_columns(pl.col('stress').log10().alias('stress_log10'))
    _tiles_.append(rt.xy(df_stress, x_field='i_global', y_field='stress_log10', dot_size=None, line_groupby_field='trial_global', color_by='round', w=1280, h=256))
#rt.tile(_tiles_, horz=False) # this fails everytime now...  bad values going into the xy component